In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from copy import deepcopy as copy
import matplotlib as plt

In [ ]:
# initial cost for a tesla, currently $39k in real life, Elon promised $25k by 2022
cost_for_tesla = 39_000
# how many miles each car can operate for before being de-commisioned
tesla_longevity_miles = 350_000
# how many miles the car is operational each month 7,500 in Tesla slides
tesla_miles_per_month = 7_500
number_of_months = 20 * 12
# 0.65 in Tesla slides
profit_per_mile = 0.65
# 0.5 in Tesla slides
occupied_miles = 0.5
# change in profit each year
profit_decay_coefficient = 0.95
# change in rate of profit decay
profit_decay_decay_exponent = 0.95
# change in tesla price each year
tesla_price_increase_coefficient = 0.03
# what order of magnitude steps up operational costs (if set to 10 will do 10, 100, 1000, etc...)
magnitude_adjustment_original = 10
current_magnitude_adjustment = magnitude_adjustment_original
# change in each tesla cost when next order of magnitude is hit (simulates operational costs for business doing acquiring)
magnitude_step_change = 0.01

In [ ]:

number_of_teslas = 1
cash = 0 - (number_of_teslas*cost_for_tesla)
remaining_miles = []
cash


In [ ]:
total_car_lifespan_months = tesla_longevity_miles//tesla_miles_per_month
last_month_miles = tesla_longevity_miles - (total_car_lifespan_months * tesla_miles_per_month)
last_month_miles


In [ ]:
car_lifespan_months_dict = {}
car_lifespan_months_dict[str((total_car_lifespan_months))] = number_of_teslas

In [ ]:
# currently represents total revenue, comment out this line and uncomment the line below to make it only money made afer debt is paid
total_money_earned = 0
# total_money_earned = cash
df= pd.DataFrame(columns=["Month", "Number of Teslas Owned", "Total Money Earned", "Cash"])
for x in range(0, number_of_months):
    print(x)
    # annual adjustment
    if x/12 == x//12 and x>0:
        profit_per_mile = profit_per_mile * profit_decay_coefficient
        profit_decay_coefficient = profit_decay_coefficient**profit_decay_decay_exponent
        print(f"Profit per mile has fallen to {profit_per_mile}")
        cost_for_tesla = cost_for_tesla * (1 + tesla_price_increase_coefficient)
        print(f"Cost per Tesla has risen to {cost_for_tesla}")
    money_earned_month = 0
    for key, value in car_lifespan_months_dict.items():
        if key == "0":
            money_earned_month = money_earned_month + (last_month_miles * profit_per_mile * occupied_miles * value)
        else:
            money_earned_month = money_earned_month + (tesla_miles_per_month * profit_per_mile * occupied_miles * value)
    for a in range(0, total_car_lifespan_months + 1):
        car_lifespan_months_dict[str(a)] = car_lifespan_months_dict.get(str((a+1)), 0)
    total_money_earned = total_money_earned + money_earned_month
    cash = cash + money_earned_month
    if cash>=cost_for_tesla:
        number_of_teslas_to_buy = int(cash//cost_for_tesla)
        # print(f"Number of Teslas to buy: {number_of_teslas_to_buy}")
        cash = cash - (number_of_teslas_to_buy*cost_for_tesla)
        car_lifespan_months_dict[str((total_car_lifespan_months))] = number_of_teslas_to_buy
    while sum(car_lifespan_months_dict.values()) >= current_magnitude_adjustment:
        cost_for_tesla = cost_for_tesla * (1 + magnitude_step_change)
        current_magnitude_adjustment = current_magnitude_adjustment * magnitude_adjustment_original
        print(f"You now have over {current_magnitude_adjustment/magnitude_adjustment_original} cars and the cost per Tesla has risen to {cost_for_tesla}")
    dictionary = {
        "Month": int(x+1),
        "Number of Teslas Owned": int(sum(car_lifespan_months_dict.values())),
        "Current Profit Per Mile": profit_per_mile,
        "Current Tesla Cost": cost_for_tesla,
        "Total Money Earned": total_money_earned,
        "Cash": cash
    }
    df = df.append(dictionary, ignore_index=True)
df

df.plot(y=['Total Money Earned', '', x="Month")

In [ ]:
df.plot(y=['Total Money Earned', 'Number of Teslas Owned', "Current Profit Per Mile", "Current Tesla Cost"], x="Month", subplots=True, sharex=True)

df.plot(y='', x="Month")

In [ ]:
# df.to_csv('robotaxi_analysis_twitter_vector.csv', index=False)